In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

df = kagglehub.dataset_load(adapter=KaggleDatasetAdapter.PANDAS, handle="fireballbyedimyrnmom/us-counties-covid-19-dataset", path='us-counties.csv')

df = df.drop('fips', axis=1)

c:\Users\olegg\OneDrive\Desktop\fluffy-funicular\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 99.9M/99.9M [00:09<00:00, 10.7MB/s]


In [3]:
df['date'] = pd.to_datetime(df['date'],  errors='coerce')
df['county'], df['state'] = df['county'].astype('string'), df['state'].astype('string')


In [2]:
df.empty

False

In [5]:
extras = pd.read_csv("./data/exstra_data.csv")
new_dataframe = pd.merge(df, extras, on='state', how='left')
new_dataframe.dtypes

date                 datetime64[ns]
county               string[python]
state                        object
cases                         int64
deaths                      float64
population                    int64
density_per_sq_mi           float64
ISO                          object
dtype: object

In [6]:
df.isnull().sum()

date          0
county        0
state         0
cases         0
deaths    57605
dtype: int64

In [8]:
population = pd.read_csv('data/population_data.csv')

In [9]:
df = df.merge(population, on='state', how='right')

In [10]:
df['population'].isnull().sum()

np.int64(0)

In [ ]:
states_codes = pd.read_csv("data/state_code.csv")
states_codes.set_index("state")

In [ ]:
population.set_index('state')
population = population.join(states_codes, how='left', lsuffix='_left', rsuffix='_right')

KeyError: "None of ['state'] are in the columns"

In [20]:
population[population['state_right'].isnull()==True]

,state_left,population,Unnamed: 0,ISO,state_right
50,Wyoming,576851,NaN,NaN,NaN


In [ ]:
population["state_left"].unique()
len(set(population["state_left"].unique()))


51

In [ ]:
import boto3
from io import BytesIO


aws_access_key_id = 'QOuSfMAuVOStCpyrUgmq'
aws_secret_access_key = 'dr5jwQf7qf50yTCwdR96FLPgoFV5hR2JOc9mNv3M'
minio_endpoint = 'http://localhost:9000'
bucket_name = 'covid'
s3 = boto3.client('s3', endpoint_url=minio_endpoint,
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)
parquet_buffer = BytesIO()
df.to_parquet(parquet_buffer, engine='pyarrow')

s3.put_object(Bucket=bucket_name,
              Key='test.parquet',
              Body=parquet_buffer.getvalue())


{'ResponseMetadata': {'RequestId': '1840395124F33FF5',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"ed1f3bc6808488c20e931e1508252055"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-checksum-crc32': 'mAg5jQ==',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '1840395124F33FF5',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '1978',
   'x-ratelimit-remaining': '1978',
   'x-xss-protection': '1; mode=block',
   'date': 'Sat, 17 May 2025 05:27:56 GMT'},
  'RetryAttempts': 0},
 'ETag': '"ed1f3bc6808488c20e931e1508252055"',
 'ChecksumCRC32': 'mAg5jQ=='}

: 

In [ ]:
import boto3
from io import StringIO


aws_access_key_id = 'QOuSfMAuVOStCpyrUgmq'
aws_secret_access_key = 'dr5jwQf7qf50yTCwdR96FLPgoFV5hR2JOc9mNv3M'
minio_endpoint = 'http://localhost:9000'
bucket_name = 'covid'
s3 = boto3.client('s3', endpoint_url=minio_endpoint,
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)
response = s3.get_object(
    Bucket=bucket_name,
    Key='population_data.csv'
)
csv_content = response['Body'].read().decode('utf-8')
population = pd.read_csv(StringIO(csv_content))
population.head()

,data/population_data.csv


In [10]:
temp_file = 'temp.parquet'

s3.download_file(Bucket='covid', Key='test.parquet', Filename=temp_file)
df = pd.read_parquet(temp_file)
print(df.head())

        date     county       state  cases  deaths
0 2020-01-21  Snohomish  Washington      1     0.0
1 2020-01-22  Snohomish  Washington      1     0.0
2 2020-01-23  Snohomish  Washington      1     0.0
3 2020-01-24       Cook    Illinois      1     0.0
4 2020-01-24  Snohomish  Washington      1     0.0
